In [240]:
import networkx as nx
from gnpsdata import taskresult
import os
from gnpsdata import workflow_fbmn
import pandas as pd

In [241]:
task = "cf6e14abf5604f47b28b467a513d3532"

In [242]:
# Downloading raw data from GNPS
def download_graphml(task, output_file):
    taskresult.download_task_resultfile(task, "gnps_molecular_network_graphml/", output_file)

def get_graphml_network(task):
    taskresult.download_task_resultfile(task, "gnps_molecular_network_graphml/", "temp.graphml")

    G = nx.read_graphml("temp.graphml")

    return G

def download_quantification(task, output_file):
    taskresult.download_task_resultfile(task, "quantification_table/", output_file)

def download_metadata(task, output_file):
    taskresult.download_task_resultfile(task, "metadata_merged/", output_file)

def download_mgf(task, output_file):
    taskresult.download_task_resultfile(task, "spectra_reformatted/", output_file)
    
# Qiime2 Data
def download_qiime2(task, output_file):
    taskresult.download_task_resultfile(task, "qiime2_output/qiime2_table.qza", output_file)

def download_qiime2_manifest(task, output_file):
    taskresult.download_task_resultfile(task, "qiime2_output/qiime2_manifest.tsv", output_file)

def download_qiime2_metadata(task, output_file):
    taskresult.download_task_resultfile(task, "qiime2_output/qiime2_metadata.tsv", output_file)

In [243]:
# Download quantification and manifest
os.makedirs("../data", exist_ok=True)
download_quantification(task, "../data/quant.csv")
download_qiime2_manifest(task, "../data/manifest.csv")
# Downloading metadata
workflow_fbmn.download_metadata(task, "../data/unprocessed_metadata.tsv")

# Changing Metadata and Manifest Column name

In [244]:
#read metadata file
metadata = pd.read_csv("../data/unprocessed_metadata.tsv", sep = "\t", index_col=False)
#rename 1st column to "#OTU ID
metadata = metadata.rename(columns={"filename":"#OTU ID"})
#convert back to .tsv
metadata.to_csv('../data/metadata.tsv', sep="\t", index=False)

# Package installation

In [245]:
!pip install pandas numpy plotly scikit-learn scikit-bio pingouin kaleido ipyfilechooser nbformat

In [246]:
# importing necessary modules
import pandas as pd
import numpy as np
import os
import itertools
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.preprocessing import StandardScaler
from scipy.spatial import distance
from sklearn.decomposition import PCA
import scipy.stats as stats
import pingouin as pg
import skbio # Don't import on Windows!!
from ipyfilechooser import FileChooser
from ipywidgets import interact
import warnings

In [247]:
# Disable warnings for cleaner output, comment out for debugging
warnings.filterwarnings('ignore')

# Blank Removal

In [248]:
# Get folder with data files
result_dir = "../data/"
#Read quant.csv and metadata .tsv
ft = pd.read_csv("../data/quant.csv")
md = pd.read_csv("../data/metadata.tsv", sep = "\t").set_index("#OTU ID")

# When cutoff is low, more noise (or background) detected; With higher cutoff, less background detected, thus more features observed
cutoff = 0.1

def inside_levels(df):
    # get all the columns (equals all attributes) -> will be number of rows
    levels = []
    types = []
    count = []
    for col in df.columns:
        types.append(type(df[col][0]))
        levels.append(sorted(set(df[col].dropna())))
        tmp = df[col].value_counts()
        count.append([tmp[levels[-1][i]] for i in range(len(levels[-1]))])
    return pd.DataFrame({"ATTRIBUTES": df.columns, "LEVELS": levels, "COUNT":count, "TYPES": types}, index=range(1, len(levels)+1))
new_md = md.copy() #storing the files under different names to preserve the original files
# remove the (front & tail) spaces, if any present, from the rownames of md
new_md.index = [name.strip() for name in md.index]
# for each col in new_md
# 1) removing the spaces (if any)
# 2) replace the spaces (in the middle) to underscore
# 3) converting them all to UPPERCASE
for col in new_md.columns:
    if new_md[col].dtype == str:
        new_md[col] = [item.strip().replace(" ", "_").upper() for item in new_md[col]]

new_ft = ft.copy() #storing the files under different names to preserve the original files
# changing the index in feature table to contain m/z and RT information
new_ft.index = [f"{id}_{round(mz, 3)}_{round(rt, 3)}" for id, mz, rt in zip(ft["row ID"], ft["row m/z"], ft["row retention time"])]
new_ft.index.name = "CustomIndex"
# drop all columns that are not mzML or mzXML file names
new_ft.drop(columns=[col for col in new_ft.columns if ".mz" not in col], inplace=True)
# remove " Peak area" from column names
new_ft.rename(columns={col: col.replace(" Peak area", "").strip() for col in new_ft.columns}, inplace=True)

if sorted(new_ft.columns) != sorted(new_md.index):
    # print the md rows / ft column which are not in ft columns / md rows and remove them
    ft_cols_not_in_md = [col for col in new_ft.columns if col not in new_md.index]
    new_ft.drop(columns=ft_cols_not_in_md, inplace=True)
    md_rows_not_in_ft = [row for row in new_md.index if row not in new_ft.columns]
    new_md.drop(md_rows_not_in_ft, inplace=True)

new_ft = new_ft.reindex(sorted(new_ft.columns), axis=1) #ordering the ft by its column names
new_md.sort_index(inplace=True) #ordering the md by its row names
list(new_ft.columns) == list(new_md.index)
data = new_md
condition = 2
df = pd.DataFrame({"LEVELS": inside_levels(data).iloc[condition-1]["LEVELS"]})
df.index = [*range(1, len(df)+1)]
#Among the shown levels of an attribute, select the ones to keep
blank_id = 1
#Splitting the data into blanks and samples based on the metadata
md_blank = data[data[inside_levels(data)['ATTRIBUTES'][condition]] == df['LEVELS'][blank_id]]
blank = new_ft[list(md_blank.index)]
md_samples = data[data[inside_levels(data)['ATTRIBUTES'][condition]] != df['LEVELS'][blank_id]]
samples = new_ft[list(md_samples.index)]

blank_removal = samples.copy()

# Getting mean for every feature in blank and Samples
avg_blank = blank.mean(axis=1, skipna=False) # set skipna = False do not exclude NA/null values when computing the result.
avg_samples = samples.mean(axis=1, skipna=False)

# Getting the ratio of blank vs samples
ratio_blank_samples = (avg_blank+1)/(avg_samples+1)

# Create an array with boolean values: True (is a real feature, ratio<cutoff) / False (is a blank, background, noise feature, ratio>cutoff)
is_real_feature = (ratio_blank_samples<cutoff)
blank_removal = samples[is_real_feature.values]
imputation_samples = blank_removal.copy()

# save to file
entry_id = []
entry_mz = []
entry_time = []
for entryCol in blank_removal.index:
    entry = entryCol.split("_")
    entry_id.append(entry[0])
    entry_mz.append(entry[1])
    entry_time.append(entry[2])
blank_removal.insert(0,"#OTU ID",entry_id,True)
blank_removal.insert(1,"sample_name",entry_mz,True)
blank_removal.insert(2,"abundance",entry_time,True)
blank_removal.to_csv(os.path.join(result_dir, "Blanks_Removed.tsv"), sep = "\t", index = False)

# Imputation

In [249]:
bins, bins_label, a = [-1, 0, 1, 10], ['-1','0', "1", "10"], 2

while a<=10:
    bins_label.append(np.format_float_scientific(10**a))
    bins.append(10**a)
    a+=1

freq_table = pd.DataFrame(bins_label)
frequency = pd.DataFrame(np.array(np.unique(np.digitize(imputation_samples.to_numpy(), bins, right=True), return_counts=True)).T).set_index(0)
freq_table = pd.concat([freq_table,frequency], axis=1).fillna(0).drop(0)
freq_table.columns = ['intensity', 'Frequency']
freq_table['Log(Frequency)'] = np.log(freq_table['Frequency']+1)

# get the lowest intensity (that is not zero) as a cutoff LOD value
cutoff_LOD = round(imputation_samples.replace(0, np.nan).min(numeric_only=True).min())

imputed = imputation_samples.copy()
entry_id = []
entry_mz = []
entry_time = []
for entryCol in imputed.index:
    entry = entryCol.split("_")
    entry_id.append(entry[0])
    entry_mz.append(entry[1])
    entry_time.append(entry[2])
imputed.insert(0,"#OTU ID",entry_id,True)
imputed.insert(1,"sample_name",entry_mz,True)
imputed.insert(2,"abundance",entry_time,True)
imputed = imputed.apply(lambda x: [np.random.randint(0, cutoff_LOD) if v == 0 else v for v in x])
# save to file
imputed.to_csv(os.path.join(result_dir, "Imputed_QuantTable.tsv"), sep = "\t", index = False)

# Normalization

In [250]:
normalized = imputation_samples.copy()
# Dividing each element of a particular column with its column sum
normalized = normalized.apply(lambda x: x/np.sum(x), axis=0)
normalized_samples = normalized.copy()
entry_id = []
entry_mz = []
entry_time = []
for entryCol in normalized_samples.index:
    entry = entryCol.split("_")
    entry_id.append(entry[0])
    entry_mz.append(entry[1])
    entry_time.append(entry[2])
normalized_samples.insert(0,"#OTU ID",entry_id,True)
normalized_samples.insert(1,"sample_name",entry_mz,True)
normalized_samples.insert(2,"abundance",entry_time,True)
normalized_samples.to_csv(os.path.join(result_dir, "Normalised_Quant_table.tsv"), sep = "\t", index = False)

# Scaling

In [251]:
# transposing the imputed table before scaling
transposed = imputation_samples.T
# put the rows in the feature table and metadata in the same order
transposed.sort_index(inplace=True)
md_samples.sort_index(inplace=True)

if (md_samples.index == transposed.index).all():
    pass
else:
    print("WARNING: Sample names in feature and metadata table are NOT the same!")
transposed.to_csv(os.path.join(result_dir, "Imputed_QuantTable_transposed.csv"))

# scale filtered data
scaled = pd.DataFrame(StandardScaler().fit_transform(transposed), index=transposed.index, columns=transposed.columns)
scaled = scaled.T
entry_id = []
entry_mz = []
entry_time = []
for entryCol in scaled.index:
    entry = entryCol.split("_")
    entry_id.append(entry[0])
    entry_mz.append(entry[1])
    entry_time.append(entry[2])
scaled.insert(0,"#OTU ID",entry_id,True)
scaled.insert(1,"sample_name",entry_mz,True)
scaled.insert(2,"abundance",entry_time,True)
scaled.to_csv(os.path.join(result_dir, "Imputed_Scaled_QuantTable.tsv"), sep = "\t", index = False)

# Import Into Qiime2
## Convert .tsv to .biom
! source activate qiime2-2019.4 && biom convert -i ../data/gnps_quant/quant.tsv -o quant.biom --to-hdf5

In [252]:
! source activate qiime2-2019.4 && biom convert \
  -i ../data/Normalised_Quant_table.tsv \
  -o ../data/quant.biom --to-hdf5

In [253]:
! source activate qiime2-2019.4 && qiime tools import \
  --input-path ../data/quant.biom \
  --type 'FeatureTable[Frequency]' \
  --input-format BIOMV210Format \
  --output-path ../data/qiime_table.qza

/opt/conda/envs/qiime2-2019.4/lib/python3.6/site-packages/skbio/util/_testing.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as pdt
Imported ../data/quant.biom as BIOMV210Format to ../data/qiime_table.qza


# Merging Metadata and Normalized Data 

In [254]:
transposed_scaled = scaled.transpose()
display(transposed_scaled.head())
display(md_samples.head())
Data = pd.merge(md_samples, transposed_scaled, left_index=True, right_index=True, how="inner")
Data.index.name = 'sample_name'
Data.to_csv(os.path.join(result_dir, "merged_metadata.tsv"), sep = "\t", index = True)

CustomIndex,2127_151.096_0.986,86368_151.098_11.096,90458_151.098_12.344,88889_151.098_11.826,87841_151.098_11.55,92531_152.947_13.15,92590_152.947_13.511,1531_152.947_0.753,39078_153.091_5.457,22910_153.091_4.085,...,88116_1444.398_11.482,89487_1444.398_12.017,86967_1444.398_11.216,90591_1444.399_12.387,91218_1444.399_12.614,92162_1444.399_12.973,88518_1444.399_11.718,88057_1445.398_11.541,89348_1445.398_11.988,91876_1445.399_12.863
#OTU ID,2127,86368,90458,88889,87841,92531,92590,1531,39078,22910,...,88116,89487,86967,90591,91218,92162,88518,88057,89348,91876
sample_name,151.096,151.098,151.098,151.098,151.098,152.947,152.947,152.947,153.091,153.091,...,1444.398,1444.398,1444.398,1444.399,1444.399,1444.399,1444.399,1445.398,1445.398,1445.399
abundance,0.986,11.096,12.344,11.826,11.55,13.15,13.511,0.753,5.457,4.085,...,11.482,12.017,11.216,12.387,12.614,12.973,11.718,11.541,11.988,12.863
SD_01-2018_10_a.mzXML,-0.36621,-0.146293,-0.672484,-0.57864,-0.506848,-1.059521,-0.730349,-0.268576,-0.843086,-0.402539,...,1.508335,3.052098,0.572418,0.252387,2.482082,-0.366943,1.031651,2.338755,0.588116,1.222231
SD_01-2018_10_b.mzXML,0.137203,-0.146293,-0.672484,-0.57864,-0.506848,-0.470587,-0.563991,-0.268576,-0.734772,-0.389619,...,4.49691,1.195304,0.879027,3.887736,0.67784,1.303225,2.430627,1.951121,-0.37495,5.024339


,ATTRIBUTE_Sample-Type,ATTRIBUTE_Batch,ATTRIBUTE_Month,ATTRIBUTE_Year,ATTRIBUTE_Sample_Location,ATTRIBUTE_Replicate,ATTRIBUTE_Spot,ATTRIBUTE_Latitude,ATTRIBUTE_Longitude,ATTRIBUTE_Sample_Area,ATTRIBUTE_Spot_Name
SD_01-2018_10_a.mzXML,Sample,2,Jan,2018,10,a,10,32.86261,-117.26042,SIO_La_Jolla_Shores,SIO_South_Pier
SD_01-2018_10_b.mzXML,Sample,2,Jan,2018,10,b,10,32.86261,-117.26042,SIO_La_Jolla_Shores,SIO_South_Pier
SD_01-2018_11_a.mzXML,Sample,2,Jan,2018,11,a,11,32.85601,-117.26253,SIO_La_Jolla_Shores,La_Jolla_Shores
SD_01-2018_11_b.mzXML,Sample,2,Jan,2018,11,b,11,32.85601,-117.26253,SIO_La_Jolla_Shores,La_Jolla_Shores
SD_01-2018_12_a.mzXML,Sample,2,Jan,2018,12,a,12,32.85161,-117.26965,La_Jolla_Cove,Cove


# ANOVA
! source activate qiime2-2019.4 && qiime longitudinal anova \
  --m-metadata-file <path-to-metadata-file.tsv> \
  --p-formula "independent_variable~dependent_variable+dependent_variable" \
  --o-visualization <path-to-visualization.qza>

In [255]:
!source activate qiime2-2019.4 && qiime longitudinal anova \
  --m-metadata-file ../data/metadata.tsv \
  --p-formula "ATTRIBUTE_Sample_Location~ATTRIBUTE_Sample_Area+ATTRIBUTE_Latitude" \
  --p-sstype 'I' \
  --o-visualization ../data/metadata.qzv

/opt/conda/envs/qiime2-2019.4/lib/python3.6/site-packages/skbio/util/_testing.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as pdt
Saved Visualization to: ../data/metadata.qzv


# Visualization
Qiime2 visualizations do not work in headless environments, we can view them at https://view.qiime2.org/

# Principal Coordinate Analysis (PCoA) & Distance Matrix
! source activate qiime2-2019.4 && first create distance matrix 
 qiime diversity beta \
  --i-table <path-to-feature-table.qza> \
  --p-metric canberra_adkins \
  --output-dir <path-to-distance-matrix.qza> \

In [256]:
! source activate qiime2-2019.4 && qiime diversity beta \
  --i-table ../data/qiime_table.qza \
  --p-metric canberra_adkins \
  --o-distance-matrix ../data/distance_matrix.qza

/opt/conda/envs/qiime2-2019.4/lib/python3.6/site-packages/skbio/util/_testing.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as pdt
Saved DistanceMatrix to: ../data/distance_matrix.qza


## PCoA
! source activate qiime2-2019.4 && qiime diversity pcoa \
  --i-distance-matrix <path-to-distance-matrix.qza> \
  --p-number-of-dimensions INTEGER \ #optional
  --o-pcoa <path-to-artifact.qza>

In [257]:
! source activate qiime2-2019.4 && qiime diversity pcoa \
  --i-distance-matrix ../data/distance_matrix.qza \
  --o-pcoa ../data/pcoa.qza

/opt/conda/envs/qiime2-2019.4/lib/python3.6/site-packages/skbio/util/_testing.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as pdt
Saved PCoAResults to: ../data/pcoa.qza


# Emperor plot
! source activate qiime2-2019.4 && qiime emperor plot \
--i-pcoa <path_to_pcoa.qza> \
--m-metadata-file <path_to_metadata.tsv \
--o-visualization emperor_qiime2

In [258]:
! source activate qiime2-2019.4 && qiime emperor plot \
  --i-pcoa ../data/pcoa.qza \
  --m-metadata-file ../data/metadata.tsv \
  --o-visualization ../data/emperor_plot \
  --p-ignore-missing-samples

/opt/conda/envs/qiime2-2019.4/lib/python3.6/site-packages/skbio/util/_testing.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as pdt
Saved Visualization to: ../data/emperor_plot.qzv
